<a href="https://colab.research.google.com/github/swsnu/swpp2019-team21/blob/ML/ML_suggest_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install konlpy       # Python 3.x
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.1.tar.gz
!tar zxfv mecab-0.996-ko-0.9.1.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.1')
!./configure
!make
!make check
!make install

import os
os.chdir('/tmp/')

 
!wget -O m4-1.4.9.tar.gz http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install

os.chdir('/tmp')
!curl -OL http://ftp.gnu.org/gnu/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

os.chdir('/tmp')
!curl -LO http://ftp.gnu.org/gnu/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

import os
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
!tar -zxvf mecab-ko-dic-2.0.1-20150920.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!ldconfig
!ldconfig -p | grep /usr/local/lib
!./autogen.sh
!./configure
!make
# !sh -c 'echo "dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic" > /usr/local/etc/mecabrc'
!make install

import os
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

from konlpy.tag import Mecab
m = Mecab()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1380k  100 1380k    0     0   497k      0  0:00:02  0:00:02 --:--:--  911k
mecab-0.996-ko-0.9.1/
mecab-0.996-ko-0.9.1/config.h.in
mecab-0.996-ko-0.9.1/Makefile.train
mecab-0.996-ko-0.9.1/ChangeLog
mecab-0.996-ko-0.9.1/configure
mecab-0.996-ko-0.9.1/swig/
mecab-0.996-ko-0.9.1/swig/version.h.in
mecab-0.996-ko-0.9.1/swig/Makefile
mecab-0.996-ko-0.9.1/swig/version.h
mecab-0.996-ko-0.9.1/swig/MeCab.i
mecab-0.996-ko-0.9.1/mkinstalldirs
mecab-0.996-ko-0.9.1/mecab-config.in
mecab-0.996-ko-0.9.1/configure.in
mecab-0.996-ko-0.9.1/mecab.iss.in
mecab-0.996-ko-0.9.1/aclocal.m4
mecab-0.996-ko-0.9.1/mecabrc.in
mecab-0.996-ko-0.9.1/INSTALL
mecab-0.996-ko-0.9.1/AUTHORS
mecab-0.996-ko-0.9.1/example/
mecab-0.996-ko-0.9.1/example/example_lattice.c
mecab-0.996-ko-0.9.1/

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np, pandas as pd
import pickle, os, re, glob
import gensim
import glob
import konlpy
import nltk
from konlpy.tag import Mecab
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import random, logging

In [0]:
os.chdir('/content/drive/My Drive/SWPP_ML/')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

tokenizer = Mecab()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [11]:
model = gensim.models.Word2Vec.load('ko.bin')

data = glob.glob('./data/*')

#############################################################################################################
#############################################################################################################

raw = {}
raw['KOR'] = []
raw['ENG'] = []

for dat in data:
    for col in ['KOR', 'ENG']:
        d = pd.read_excel(dat)
        print(dat, d.columns)
        raw[col] += list(d[col])

dat = pd.DataFrame(raw)

print("Data Loaded")
print("#"*100)
#############################################################################################################
#############################################################################################################

dat.KOR = dat.KOR.apply(tokenizer.pos)
dat.ENG = dat.ENG.apply(word_tokenize)
dat.ENG = dat.ENG.apply(pos_tag)

stops = set(stopwords.words('english'))

print("Processing on Korean Data")

for i in range(len(dat)):
    dat.KOR[i] = list(map(lambda x : x[0], list(filter(lambda x : x[1] in ['NNG', 'NNP'], dat.KOR[i]))))

print("Processing on English Data")

for i in range(len(dat)):
    dat.ENG[i] = list(map(lambda x : x[0], list(filter(lambda x : x[1] in ['NN', 'NNP'], dat.ENG[i]))))

print("Data Preprocessed")
print("#"*100)
#############################################################################################################
#############################################################################################################

ipt = []

print("Merging Sentences")

for i in range(len(dat)):
    ipt.append(dat.KOR[i] + dat.ENG[i])

for i in range(len(ipt)):
    random.shuffle(ipt[i])

print("Input Set")
print("#"*100)
#############################################################################################################
#############################################################################################################

model.build_vocab(sentences=ipt, update=True, min_count=20)
model.train(sentences=ipt, epochs=10, total_examples=len(ipt))

model.save('models_engadd.bin')

print("Done!")
print("#"*100)


2019-11-25 09:51:53,653 : INFO : loading Word2Vec object from ko.bin
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-11-25 09:51:53,831 : INFO : setting ignored attribute syn0norm to None
2019-11-25 09:51:53,832 : INFO : setting ignored attribute cum_table to None
2019-11-25 09:51:53,835 : INFO : Model saved using code from earlier Gensim Version. Re-loading old model in a compatible way.
2019-11-25 09:51:53,837 : INFO : loading Word2Vec object from ko.bin
2019-11-25 09:51:54,104 : INFO : setting ignored attribute syn0norm to None
2019-11-25 09:51:54,105 : INFO : setting ignored attribute cum_table to None
2019-11-25 09:51:54,108 : INFO : loaded ko.bin


./data/6.문어체-지자체웹사이트.xlsx Index(['ID', '제목', 'URL', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/6.문어체-지자체웹사이트.xlsx Index(['ID', '제목', 'URL', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/1.구어체.xlsx Index(['mid_sid', 'KOR', 'ENG'], dtype='object')
./data/1.구어체.xlsx Index(['mid_sid', 'KOR', 'ENG'], dtype='object')
./data/4.문어체-한국문화.xlsx Index(['ID', '분야', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/4.문어체-한국문화.xlsx Index(['ID', '분야', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/5.문어체-조례.xlsx Index(['ID', '분야', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/5.문어체-조례.xlsx Index(['ID', '분야', 'KOR', 'NMT', 'ENG'], dtype='object')
./data/3.문어체-뉴스-2.xlsx Index(['id', '날짜', '자동분류_1', '자동분류_2', '자동분류_3', 'URL', '언론사', 'KOR', 'ENG'], dtype='object')
./data/3.문어체-뉴스-2.xlsx Index(['id', '날짜', '자동분류_1', '자동분류_2', '자동분류_3', 'URL', '언론사', 'KOR', 'ENG'], dtype='object')
./data/2.대화체.xlsx Index(['대분류', '소분류', '상황', 'Set Nr.', '

2019-11-25 10:07:18,876 : INFO : collecting all words and their counts
2019-11-25 10:07:18,880 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-25 10:07:18,909 : INFO : PROGRESS: at sentence #10000, processed 61372 words, keeping 11428 word types
2019-11-25 10:07:18,928 : INFO : PROGRESS: at sentence #20000, processed 104786 words, keeping 15337 word types
2019-11-25 10:07:18,949 : INFO : PROGRESS: at sentence #30000, processed 149724 words, keeping 18801 word types
2019-11-25 10:07:18,969 : INFO : PROGRESS: at sentence #40000, processed 193093 words, keeping 21254 word types
2019-11-25 10:07:18,991 : INFO : PROGRESS: at sentence #50000, processed 240309 words, keeping 23793 word types
2019-11-25 10:07:19,014 : INFO : PROGRESS: at sentence #60000, processed 292314 words, keeping 26939 word types
2019-11-25 10:07:19,035 : INFO : PROGRESS: at sentence #70000, processed 340012 words, keeping 29034 word types
2019-11-25 10:07:19,076 : INFO : PROGRESS: at s

Input Set
####################################################################################################


2019-11-25 10:07:19,153 : INFO : PROGRESS: at sentence #90000, processed 683484 words, keeping 43602 word types
2019-11-25 10:07:19,227 : INFO : PROGRESS: at sentence #100000, processed 906322 words, keeping 45301 word types
2019-11-25 10:07:19,317 : INFO : PROGRESS: at sentence #110000, processed 1161022 words, keeping 62400 word types
2019-11-25 10:07:19,341 : INFO : PROGRESS: at sentence #120000, processed 1217547 words, keeping 63126 word types
2019-11-25 10:07:19,366 : INFO : PROGRESS: at sentence #130000, processed 1273636 words, keeping 64003 word types
2019-11-25 10:07:19,463 : INFO : PROGRESS: at sentence #140000, processed 1533840 words, keeping 75175 word types
2019-11-25 10:07:19,562 : INFO : PROGRESS: at sentence #150000, processed 1809568 words, keeping 84980 word types
2019-11-25 10:07:19,652 : INFO : PROGRESS: at sentence #160000, processed 2059052 words, keeping 93114 word types
2019-11-25 10:07:19,739 : INFO : PROGRESS: at sentence #170000, processed 2305037 words, ke

Done!
####################################################################################################
